To create the data set file by scrapping nltk treebank tag annonated text. Two data files "treebank_sents.txt" and "treebank_poss.txt" to store sentences and related pos tags correspondingly.

In [1]:
import nltk
import os
def get_tagging_data(DATA_DIR):
    open(os.path.join(DATA_DIR, "treebank_sents.txt"), "w").close()
    open(os.path.join(DATA_DIR, "treebank_poss.txt"), "w").close()
    fedata = open(os.path.join(DATA_DIR, "treebank_sents.txt"), "a")
    ffdata = open(os.path.join(DATA_DIR, "treebank_poss.txt"), "a")
    sents = nltk.corpus.treebank.tagged_sents()
    for sent in sents:
        words, poss = [], []
        for word, pos in sent:
            if pos == "-NONE-":
                continue
            words.append(word)
            poss.append(pos)
        fedata.write(" ".join(words))
        fedata.write('\n')
        ffdata.write(" ".join(poss))
        ffdata.write('\n')
    fedata.close()
    ffdata.close()

DATA_DIR= os.getcwd()
get_tagging_data(DATA_DIR)
    

Importing all the necessary packages and fucntions needed for the model

In [2]:
from keras.layers.core import Activation,Dense,Dropout,RepeatVector,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import numpy as np



/Users/gowthamkannan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/gowthamkannan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


To build a dictionary for words and tags

In [3]:
def parse_sentences(filename):
    word_freqs = collections.Counter()
    num_recs, maxlen = 0, 0
    fin = open(filename, "rb")
    for line in fin:
        words = line.strip().lower().split()
        for word in words:
            word_freqs[word] += 1
        if len(words) > maxlen:
            maxlen = len(words)
        num_recs += 1
    fin.close()
    return word_freqs, maxlen, num_recs

s_wordfreqs, s_maxlen, s_numrecs = parse_sentences(os.path.join(DATA_DIR, "treebank_sents.txt"))
t_wordfreqs, t_maxlen, t_numrecs = parse_sentences(os.path.join(DATA_DIR, "treebank_poss.txt"))

Creating a word2index and index2word dicitonary for the embedding layer to use. Also definig max sequence length(max sentence length that we are dealing with),the number of pos tags(nos. of labels for teh netowork)

In [4]:
MAX_SEQ_LEN=250
S_MAX_FEATURES=5000
T_MAX_FEATURES=45
S_VOCAB_SIZE=min(len(s_wordfreqs),S_MAX_FEATURES)+2
s_word2index={x[0]:i+2 for i,x in enumerate(s_wordfreqs.most_common(S_MAX_FEATURES))}
s_word2index['PAD']=0
s_word2index['UNK']=1
s_index2word={v:k for k,v in s_word2index.items()}
t_vocabsize=len(t_wordfreqs)+1
t_word2index={x[0]:i for i,x in enumerate(t_wordfreqs.most_common(T_MAX_FEATURES))}
t_word2index['PAD']=0
t_index2word={v:k for k,v in t_word2index.items()}



Building the input and output  tensor for the neural network

In [10]:
def construct_tensor(filename,numrecs,word2index,max_len,make_categorical=False,num_classes=0):
    data=np.empty((numrecs,),dtype=list)
    fin=open(filename,"rb")
    i=0
    for line in fin:
        wids=[]
        for word in line.strip().lower().split():
            if word in word2index.keys():
                wids.append(word2index[word])
            else:
                wids.append(word2index['UNK'])
            if make_categorical:
                data[i]=np_utils.to_categorical(wids,num_classes=num_classes)
            else:
                data[i]=wids
        i=i+1
    fin.close()

    pdata=sequence.pad_sequences(data,maxlen=max_len)
    return pdata

X=construct_tensor(os.path.join(DATA_DIR,"treebank_sents.txt"),s_numrecs,s_word2index,MAX_SEQ_LEN)
Y=construct_tensor(os.path.join(DATA_DIR,"treebank_poss.txt"),t_numrecs,t_word2index,MAX_SEQ_LEN,True,t_vocabsize)
    

Spiting the dataset for training and testing.

In [11]:
X_Train,X_Test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)


Intializing the reccurent neural network with the necessary activation functions etc. Also specifiy the size of the emebedding , output of the embedding layer. 

In [12]:
EMBED_SIZE=128
HIDDEN_LAYER_SIZE=50
BATCH_SIZE=32
NUM_EPOCHS=2
model=Sequential()
model.add(Embedding(S_VOCAB_SIZE,EMBED_SIZE,input_length=MAX_SEQ_LEN))
model.add(SpatialDropout1D(0.2))
model.add(GRU(HIDDEN_LAYER_SIZE,dropout=0.0,recurrent_dropout=0.0))
model.add(RepeatVector(MAX_SEQ_LEN))
model.add(GRU(HIDDEN_LAYER_SIZE,dropout=0.0,recurrent_dropout=0.0,return_sequences=True))
model.add(TimeDistributed(Dense(t_vocabsize)))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


Training the neural network.

In [13]:
history=model.fit(X_Train,y_train,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,validation_data=[X_Test,y_test])
score,accuracy=model.evaluate(X_Test,y_test,batch_size=BATCH_SIZE)
print("Test score:%.3f,accuracy:%.3f" %(score,accuracy))

Train on 3131 samples, validate on 783 samples
Epoch 1/2
3131/3131 [==============================] - 72s 23ms/step - loss: 0.3032 - acc: 0.6989 - val_loss: 0.2933 - val_acc: 0.9146
Epoch 2/2
783/783 [==============================] - 3s 4ms/step
Test score:0.291,accuracy:0.911


In [ ]:
Ploting the performance of the model during trainig phase.

In [50]:
import matplotlib.pyplot as plt
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="Train")
plt.plot(history.history["val_acc"], color="b", label="Validation")
plt.legend(loc="best")
plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

Sample test case

In [49]:
text="how are you"
data=np.empty((1,),dtype=list)
wids=[]
for word in text.split():
    if word in s_word2index.keys():
        wids.append(s_word2index[word])
    else:
        wids.append(s_word2index['UNK'])
data[0]=wids
new_data=sequence.pad_sequences(data,maxlen=MAX_SEQ_LEN)
y=model.predict(new_data)


